In [2]:
# start ollama server
# !ollama serve

In [40]:
# download to:  ~/.ollama/models
#!ollama pull nomic-embed-text
# !ollama pull llama3.1:8b
!ollama list

NAME                       ID              SIZE      MODIFIED      
llama3.1:8b                42182419e950    4.7 GB    6 minutes ago    
nomic-embed-text:latest    0a109f422b47    274 MB    5 months ago     


In [117]:
#!ollama help
# !ollama rm llama3:latest
#ollama run llama3.1

In [4]:
import os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

llm = AzureChatOpenAI(
    openai_api_base="https://free-cdo.openai.azure.com/openai/deployments/cod-free-gpt4o/chat/completions?api-version=2024-02-15-preview",
    openai_api_version="2024-02-15-preview",
    openai_api_type="azure",
    temperature=0.2,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.3,
)

In [114]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class OutputStructure(BaseModel):
    input_text: str = Field(description="The input text")
    translated_text: str = Field(description="The translated text")
 
parser = JsonOutputParser(pydantic_object=OutputStructure)

In [115]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

system_template = """You are a helpful assistant that translates {input_language} to {output_language}.
{format_instructions}"""

human_template = "{text}"

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [116]:
from pprint import PrettyPrinter

prompt = chat_prompt.partial(
    format_instructions=parser.get_format_instructions(),
    input_language="English",
    output_language="German"
)

chain = prompt | llm | parser

result = chain.invoke({"text": "I love programming."})

PrettyPrinter().pprint(result) 

{'input_text': 'I love programming.',
 'translated_text': 'Ich liebe Programmieren.'}


## Graph from text example:

In [153]:
from pydantic import BaseModel, Field

class JSONSchemaSpecification(BaseModel):
    notes: str = Field(description="Any notes or comments about the schema")
    jsonschema: str = Field(description="A JSON array of JSON schema specifications that describe the entities in the data model")

parser = JsonOutputParser(pydantic_object=JSONSchemaSpecification)

In [190]:
from pathlib import Path
from langchain_community.document_loaders import TextLoader

txt_path = Path().cwd().parent / "dune.txt"

loader = TextLoader(str(txt_path))
documents = loader.load()

In [158]:
print(documents[0].page_content)

Dune is a 1965 epic science fiction novel by American author Frank Herbert, originally published as two separate serials in Analog magazine. It tied with Roger Zelazny's This Immortal for the Hugo Award in 1966 and it won the inaugural Nebula Award for Best Novel. It is the first installment of the Dune Chronicles. It is one of the world's best-selling science fiction novels.Dune is set in the distant future in a feudal interstellar society in which various noble houses control planetary fiefs.  It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or "spice", a drug that extends life and enhances mental abilities. Melange is also necessary for space navigation, which requires a kind of multidimensional awareness and foresight that only the drug provides. As melange can only be produced on Arrakis, control of the planet is a cov

In [156]:
example_output = [
   dict(
    title="Person",
    type="object",
    description="Node",
    properties=[
        dict(name="name", type="string", description="The name of the person", examples=["Tom Hanks"]),
        # dict(name="date_of_birth", column_name="person_dob", type="date", description="The date of birth for the person", examples=["1987-06-05"]),
        # dict(name="id", column_name="person_name, date_of_birth", type="string", description="The ID is a combination of name and date of birth to ensure uniqueness", examples=["tom-hanks-1987-06-05"]),
    ],
  ),
#    dict(
#     title="Director",
#     type="object",
#     description="Node",
#     properties=[
#         dict(name="name", column_name="director_names", type="string", description="The name of the directors. Split values in text by a comma", examples=["Francis Ford Coppola"]),
#     ],
#   ),
#    dict(
#     title="Movie",
#     type="object",
#     description="Node",
#     properties=[
#         dict(name="title", column_name="title", type="string", description="The title of the movie", examples=["Toy Story"]),
#         dict(name="released", column_name="released", type="integer", description="The year the movie was released", examples=["1990"]),
#     ],)
   dict(
    title="IS_FATHER_OF",
    type="object",
    description="Relationship",
    properties=[
        dict(name="_from", type="string", description="The Father", examples=["Person"]),
        dict(name="_to", type="string", description="The child", examples=["Person"]),
    ]
  )
#    dict(
#     title="DIRECTED",
#     type="object",
#     description="Relationship",
#     properties=[
#         dict(name="_from", type="string", column_name="director_names", description="Director names are comma separated", examples=["Director"]),
#         dict(name="_to", type="string", column_name="title", description="The label of the node this relationship ends at", examples=["Movie"]),
#     ],
#   ),
]

In [159]:
from langchain.prompts import PromptTemplate
from json import dumps

model_prompt = PromptTemplate.from_template("""
You are an expert Graph Database administrator.
Your task is to design a data model based on the information provided from an existing data source.

You must decide where the following text fits in with the existing data model.  Consider:
* Does the text represent an entity, for example a Person, Place, or Movie?  If so, this should be a node in its own right.
* Does the text represent a relationship between two entities?  If so, this should be a relationship between two nodes.
                                            
This is the text you need to consider: {text}

Here is an example of a good output:
{example_output}
""", partial_variables=dict(example_output=dumps(example_output)))

chain = model_prompt | llm | parser

In [160]:
result = chain.invoke({"text": documents[0].page_content})

# # Process all chunks in parallel using batch
# results = chain.batch([{"text": chunk.page_content} for chunk in documents], {"max_concurrency": 5})

# # Print the results
# for result in results:
#     PrettyPrinter().pprint(result) 

In [3]:
#PrettyPrinter().pprint(result)

In [10]:
from langchain_community.graphs import Neo4jGraph

url = "bolt://localhost:7687"  # Default local Neo4j URL
password = "neo4j_password"  # Set this to your database password

graph = Neo4jGraph(
    url=url,
    username="neo4j",
    password=password,
    database="dune-example",
)

In [11]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

In [12]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""

text = """Duke Leto Atreides of House Atreides, ruler of the ocean planet Caladan, is assigned by the Padishah Emperor Shaddam IV to serve as fief ruler of the planet Arrakis. Although Arrakis is a harsh and inhospitable desert planet, it is of enormous importance because it is the only planetary source of melange, or the "spice", a unique and incredibly valuable substance that extends human youth, vitality and lifespan. It is also through the consumption of spice that Spacing Guild Navigators are able to effect safe interstellar travel. Shaddam, jealous of Duke Leto Atreides's rising popularity in the Landsraad, sees House Atreides as a potential future rival and threat, so conspires with House Harkonnen, the former stewards of Arrakis and the longstanding enemies of House Atreides, to destroy Leto and his family after their arrival. Leto is aware his assignment is a trap of some kind, but is compelled to obey the Emperor's orders anyway.
Leto's concubine Lady Jessica is an acolyte of the Bene Gesserit, an exclusively female group that pursues mysterious political aims and wields seemingly superhuman physical and mental abilities, such as the ability to control their bodies down to the cellular level, and also decide the sex of their children. Though Jessica was instructed by the Bene Gesserit to bear a daughter as part of their breeding program, out of love for Leto she bore a son, Paul. From a young age, Paul has been trained in warfare by Leto's aides, the elite soldiers Duncan Idaho and Gurney Halleck. Thufir Hawat, the Duke's Mentat (human computers, able to store vast amounts of data and perform advanced calculations on demand), has instructed Paul in the ways of political intrigue. Jessica has also trained her son in Bene Gesserit disciplines.
Paul's prophetic dreams interest Jessica's superior, the Reverend Mother Gaius Helen Mohiam, who subjects Paul to the deadly gom jabbar test. Holding a poisonous needle to his neck ready to strike should he be unable to resist the impulse to withdraw his hand from the nerve induction box, she tests Paul's self-control to overcome the extreme psychological pain he is being subjected to through the box.
Leto, Jessica, and Paul travel with their household to occupy Arrakeen, the capital on Arrakis formerly held by House Harkonnen. Leto learns of the dangers involved in harvesting the spice, which is protected by giant sandworms, and seeks to negotiate with the planet's native Fremen people, seeing them as a valuable ally rather than foes. Soon after the Atreides's arrival, Harkonnen forces attack, joined by the Emperor's ferocious Sardaukar troops in disguise. Leto is betrayed by his personal physician, the Suk doctor Wellington Yueh, who delivers a drugged Leto to the Baron Vladimir Harkonnen and his twisted Mentat, Piter De Vries. Yueh, however, arranges for Jessica and Paul to escape into the desert, where they are presumed dead by the Harkonnens. Yueh replaces one of Leto's teeth with a poison gas capsule, hoping Leto can kill the Baron during their encounter. The Baron narrowly avoids the gas due to his shield, which kills Leto, De Vries, and the others in the room. The Baron forces Hawat to take over De Vries's position by dosing him with a long-lasting, fatal poison and threatening to withhold the regular antidote doses unless he obeys. While he follows the Baron's orders, Hawat works secretly to undermine the Harkonnens.
Having fled into the desert, Paul is exposed to high concentrations of spice and has visions through which he realizes he has significant powers (as a result of the Bene Gesserit breeding scheme). He foresees potential futures in which he lives among the planet's native Fremen before leading them on a Holy Jihad across the known universe.
It is revealed Jessica is the daughter of Baron Harkonnen, a secret kept from her by the Bene Gesserit. After being captured by Fremen, Paul and Jessica are accepted into the Fremen community of Sietch Tabr, and teach the Fremen the Bene Gesserit fighting technique known as the "weirding way". Paul proves his manhood by killing a Fremen named Jamis in a ritualistic crysknife fight and chooses the Fremen name Muad'Dib, while Jessica opts to undergo a ritual to become a Reverend Mother by drinking the poisonous Water of Life. Pregnant with Leto's daughter, she inadvertently causes the unborn child, Alia, to become infused with the same powers in the womb. Paul takes a Fremen lover, Chani, and has a son with her, Leto II.
Two years pass and Paul's powerful prescience manifests, which confirms for the Fremen that he is their prophesied messiah, a legend planted by the Bene Gesserit's Missionaria Protectiva. Paul embraces his father's belief that the Fremen could be a powerful fighting force to take back Arrakis, but also sees that if he does not control them, their jihad could consume the entire universe. Word of the new Fremen leader reaches both Baron Harkonnen and the Emperor as spice production falls due to their increasingly destructive raids. The Baron encourages his brutish nephew Glossu Rabban to rule with an iron fist, hoping the contrast with his shrewder nephew Feyd-Rautha will make the latter popular among the people of Arrakis when he eventually replaces Rabban. The Emperor, suspecting the Baron of trying to create troops more powerful than the Sardaukar to seize power, sends spies to monitor activity on Arrakis. Hawat uses the opportunity to sow seeds of doubt in the Baron about the Emperor's true plans, putting further strain on their alliance.
Gurney, having survived the Harkonnen coup becomes a smuggler, reuniting with Paul and Jessica after a Fremen raid on his harvester. Believing Jessica to be the traitor, Gurney threatens to kill her, but is stopped by Paul. Paul did not foresee Gurney's attack, and concludes he must increase his prescience by drinking the Water of Life, which is traditionally fatal to males. Paul falls into unconsciousness for three weeks after drinking the poison, but when he wakes, he has clairvoyance across time and space: he is the Kwisatz Haderach, the ultimate goal of the Bene Gesserit breeding program.
Paul senses the Emperor and Baron are amassing fleets around Arrakis to quell the Fremen rebellion, and prepares the Fremen for a major offensive against the Harkonnen troops. The Emperor arrives with the Baron on Arrakis. The Emperor's troops seize a Fremen outpost, killing many including young Leto II, while Alia is captured and taken to the Emperor. Under cover of an electric storm, which shorts out the Emperor's troops' defensive shields, Paul and the Fremen, riding giant sandworms, assault the capital while Alia assassinates the Baron and escapes. The Fremen quickly defeat both the Harkonnen and Sardaukar troops.
Paul faces the Emperor, threatening to destroy spice production forever unless Shaddam abdicates the throne. Feyd-Rautha attempts to stop Paul by challenging him to a ritualistic knife fight, during which he attempts to cheat and kill Paul with a poison spur in his belt. Paul gains the upper hand and kills him. The Emperor reluctantly cedes the throne to Paul and promises his daughter Princess Irulan's hand in marriage. As Paul takes control of the Empire, he realizes that while he has achieved his goal, he is no longer able to stop the Fremen jihad, as their belief in him is too powerful to restrain.
"""

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Duke Leto Atreides', type='Person', properties={}), Node(id='House Atreides', type='Organization', properties={}), Node(id='Caladan', type='Place', properties={}), Node(id='Padishah Emperor Shaddam Iv', type='Person', properties={}), Node(id='Arrakis', type='Place', properties={}), Node(id='Melange', type='Substance', properties={}), Node(id='Spacing Guild Navigators', type='Organization', properties={}), Node(id='House Harkonnen', type='Organization', properties={}), Node(id='Lady Jessica', type='Person', properties={}), Node(id='Bene Gesserit', type='Organization', properties={}), Node(id='Paul', type='Person', properties={}), Node(id='Duncan Idaho', type='Person', properties={}), Node(id='Gurney Halleck', type='Person', properties={}), Node(id='Thufir Hawat', type='Person', properties={}), Node(id='Reverend Mother Gaius Helen Mohiam', type='Person', properties={}), Node(id='Gom Jabbar Test', type='Event', properties={}), Node(id='Arrakeen', type='Place', properties={

In [13]:
print(text)

Duke Leto Atreides of House Atreides, ruler of the ocean planet Caladan, is assigned by the Padishah Emperor Shaddam IV to serve as fief ruler of the planet Arrakis. Although Arrakis is a harsh and inhospitable desert planet, it is of enormous importance because it is the only planetary source of melange, or the "spice", a unique and incredibly valuable substance that extends human youth, vitality and lifespan. It is also through the consumption of spice that Spacing Guild Navigators are able to effect safe interstellar travel. Shaddam, jealous of Duke Leto Atreides's rising popularity in the Landsraad, sees House Atreides as a potential future rival and threat, so conspires with House Harkonnen, the former stewards of Arrakis and the longstanding enemies of House Atreides, to destroy Leto and his family after their arrival. Leto is aware his assignment is a trap of some kind, but is compelled to obey the Emperor's orders anyway.
Leto's concubine Lady Jessica is an acolyte of the Bene 

In [14]:
llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Location", "Organization"],
    allowed_relationships=["NATIONALITY", "LIVES_IN", "SPOUSE", "RELATIVE", "IS_FATHER_OF", "IS_MOTHER_OF", "IS_SIBLING_OF"],
    node_properties=["born_year", "name", "title"],
)
graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)

print(f"Nodes:{graph_documents_props[0].nodes}")
print(f"Relationships:{graph_documents_props[0].relationships}")

Nodes:[Node(id='Duke Leto Atreides', type='Person', properties={'title': 'Duke'}), Node(id='House Atreides', type='Organization', properties={}), Node(id='Caladan', type='Location', properties={}), Node(id='Padishah Emperor Shaddam Iv', type='Person', properties={}), Node(id='Arrakis', type='Location', properties={}), Node(id='Spacing Guild Navigators', type='Organization', properties={}), Node(id='House Harkonnen', type='Organization', properties={}), Node(id='Lady Jessica', type='Person', properties={}), Node(id='Bene Gesserit', type='Organization', properties={}), Node(id='Paul Atreides', type='Person', properties={}), Node(id='Duncan Idaho', type='Person', properties={}), Node(id='Gurney Halleck', type='Person', properties={}), Node(id='Thufir Hawat', type='Person', properties={}), Node(id='Reverend Mother Gaius Helen Mohiam', type='Person', properties={}), Node(id='Baron Vladimir Harkonnen', type='Person', properties={}), Node(id='Piter De Vries', type='Person', properties={}), No

In [15]:
graph.add_graph_documents(graph_documents_props, baseEntityLabel=True)